## 챗봇 예제(역할 부여)

In [5]:
# 사전 설치 : pip install ollama
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
import warnings

In [6]:
# 챗봇의 기본적 질문, 답변 역할
def ask_gemma(question):
    # ollama를 사용하여 모델로부터 응답 생성
    chatbot_role = "You are a helpful assistant."
    response = ollama.chat(model='gemma2', messages=[
        {"role": "system", "content": chatbot_role},  # 챗봇의 기본 역할 부여
        {"role": "user", "content": question}, # 질문
    ])

    return response['message']['content']

In [7]:
question = "세계에서 가장 높은 산을 알려줘."
response = ask_gemma(question)
print(response)

ResponseError: model requires more system memory (7.7 GiB) than is available (4.5 GiB)

## 챗봇 예제(Gradio 사용)

In [8]:
# 사전 설치 : pip install gradio
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr

c:\AiProject\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# ChatOllama 모델 초기화
model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)

C:\Users\human-06\AppData\Local\Temp\ipykernel_10352\3441953186.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)


In [10]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(message, history):
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # 현재 메시지 추가
    chat_history.append(HumanMessage(content=message))

    # 모델을 사용하여 응답 생성
    response = model.invoke(chat_history)

    return response.content

In [11]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "안녕하세요!",
        "인공지능에 대해 설명해주세요.",
        "파이썬의 장점은 무엇인가요?"
    ],
    title="AI 챗봇",
    description="질문을 입력하면 AI가 답변합니다."
)

c:\AiProject\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [27]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [28]:
demo.close()

Closing server running on port: 7861


## 챗봇 예제(Gradio + csv 사용)

In [14]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [15]:
# CSV 파일 로드
df = pd.read_csv("./dataset/indata_kor.csv", encoding='CP949')

In [16]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df.to_string()))

In [17]:
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\human-06\AppData\Local\Temp\ipykernel_10352\2819514520.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\AiProject\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\human-06\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

In [19]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", tempeature=0.1)

In [20]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True,
    verbose=False
)

In [21]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]

    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [22]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

c:\AiProject\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [25]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\human-06\AppData\Local\Temp\ipykernel_10352\4160252103.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": message, "chat_history": chat_history})
Traceback (most recent call last):
  File "c:\AiProject\.venv\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\AiProject\.venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\AiProject\.venv\lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
  File "c:\AiProject\.venv\lib\site-packages\gradio\blocks.py", line 1592, in call_function
    prediction = await fn(*processed_input)
  File "c:\AiProject\.venv\lib\site-packages\gradio\utils.py", line 836, in async_wrapper
    response = await f(*

In [26]:
demo.close()

Closing server running on port: 7861
